In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import numpy as np
import pandas as pd

In [ ]:
driver = webdriver.Chrome('C:\chromedriver_win32\chromedriver.exe')
driver.maximize_window()

C:\Users\DELL\AppData\Local\Temp\ipykernel_15740\3287780619.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:\chromedriver_win32\chromedriver.exe')


In [ ]:
url = 'https://xe.chotot.com/mua-ban-oto'
driver.get(url)
skips = driver.find_element(By.XPATH, '//*[@id="__next"]/div/header/div[2]')
skips.click()

In [ ]:
columns = ['Hãng', 'Dòng xe', 'Năm sản xuất', 'Số Km đã đi', 'Tình trạng', 'Hộp số', 'Nhiên liệu',
                 'Xuất xứ', 'Kiểu dáng', 'Số chỗ', 'Chính sách bảo hành', 'Trọng lượng', 'Trọng tải', 'Giá']
print(len(columns))
rows = []

14


In [ ]:
num_products = 500
Check_num_products = True
num_pages = 1
max_features = 0

In [ ]:
while Check_num_products:
    page_source = BeautifulSoup(driver.page_source, 'html.parser')
    link_items = page_source.find_all(class_='AdItem_adItem__gDDQT')
    link_items = ['https://xe.chotot.com/' + link.get('href') for link in link_items]

    for i in range(len(link_items)):
        driver.get(link_items[i])
        page_source = BeautifulSoup(driver.page_source, 'html.parser')

        price = page_source.find(class_='AdDecriptionVeh_price__u_N83').text
        price = float(price.replace('.', '').replace('đ', '').replace('\xa0', ''))

        for k in range(4,10):
            try:
                detail_button = driver.find_element(By.XPATH, f'/html/body/div[1]/div/div[4]/div[1]/div/div[4]/div/div[{k}]/button')
                break
            except: pass
        detail_button.click()

        page_source = BeautifulSoup(driver.page_source, 'html.parser')
        body_info = page_source.find_all(class_='media-body media-middle')
        infos = [info.text for info in body_info]

        row = [np.nan for _ in range(len(columns))]
        for i in infos:
            name, value = i.split(':')
            row[columns.index(name)] = value.strip()
        row[-1] = price

        if len(infos) > max_features:
            max_features = len(infos)

        rows.append(row)
        print(f'{row}')
        if len(rows) == num_products:
            Check_num_products = False
            break

    if Check_num_products:
        num_pages += 1
        driver.get(f"https://xe.chotot.com/mua-ban-oto?page={num_pages}")

['Hyundai', 'Kona', '2018', '69000', 'Đã sử dụng', 'Tự động', 'Xăng', 'Đang cập nhật', 'SUV / Cross over', '5', 'Bảo hành hãng', '> 1 tấn', '> 2 tấn', 488000000.0]
['Kia', 'Forte', '2012', '130000', 'Đã sử dụng', 'Tự động', 'Xăng', 'Đang cập nhật', 'Sedan', '5', 'Bảo hành hãng', '> 1 tấn', '> 2 tấn', 335000000.0]
['Toyota', 'Vios', '2019', '30000', 'Đã sử dụng', 'Tự động', 'Xăng', 'Đang cập nhật', 'Sedan', '5', 'Bảo hành hãng', '> 1 tấn', '> 2 tấn', 415000000.0]
['Toyota', 'Corolla Cross', '2021', '21000', 'Đã sử dụng', 'Tự động', 'Xăng', 'Đang cập nhật', nan, nan, 'Bảo hành hãng', '> 1 tấn', '> 2 tấn', 695000000.0]
['Mitsubishi', 'Xpander', '2023', nan, 'Mới', 'Tự động', 'Xăng', 'Nước khác', 'Minivan (MPV)', '7', 'Bảo hành hãng', '> 1 tấn', '> 2 tấn', 515000000.0]
['Toyota', 'Vios', '2016', '8300', 'Đã sử dụng', 'Số sàn', 'Xăng', 'Việt Nam', 'Sedan', '2', 'Bảo hành hãng', '> 1 tấn', '> 2 tấn', 255000000.0]
['Chevrolet', 'Cruze', '2011', '1000', 'Đã sử dụng', 'Số sàn', 'Xăng', 'Hàn Quố

In [ ]:
result = pd.DataFrame(data=rows, columns=columns)
result

,Hãng,Dòng xe,Năm sản xuất,Số Km đã đi,Tình trạng,Hộp số,Nhiên liệu,Xuất xứ,Kiểu dáng,Số chỗ,Chính sách bảo hành,Trọng lượng,Trọng tải,Giá
0,Hyundai,Kona,2018,69000,Đã sử dụng,Tự động,Xăng,Đang cập nhật,SUV / Cross over,5,Bảo hành hãng,> 1 tấn,> 2 tấn,4.880000e+08
1,Kia,Forte,2012,130000,Đã sử dụng,Tự động,Xăng,Đang cập nhật,Sedan,5,Bảo hành hãng,> 1 tấn,> 2 tấn,3.350000e+08
2,Toyota,Vios,2019,30000,Đã sử dụng,Tự động,Xăng,Đang cập nhật,Sedan,5,Bảo hành hãng,> 1 tấn,> 2 tấn,4.150000e+08
3,Toyota,Corolla Cross,2021,21000,Đã sử dụng,Tự động,Xăng,Đang cập nhật,NaN,NaN,Bảo hành hãng,> 1 tấn,> 2 tấn,6.950000e+08
4,Mitsubishi,Xpander,2023,NaN,Mới,Tự động,Xăng,Nước khác,Minivan (MPV),7,Bảo hành hãng,> 1 tấn,> 2 tấn,5.150000e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Toyota,Land Cruiser Prado,2016,90000,Đã sử dụng,Tự động,Xăng,Nhật Bản,SUV / Cross over,7,Bảo hành hãng,> 1 tấn,> 2 tấn,1.380000e+09
496,Kia,Seltos,2021,46000,Đã sử dụng,Tự động,Xăng,Việt Nam,SUV / Cross over,5,Bảo hành hãng,> 1 tấn,> 2 tấn,5.380000e+08
497,Toyota,Vios,2013,120000,Đã sử dụng,Số sàn,Xăng,Việt Nam,Sedan,5,Bảo hành hãng,> 1 tấn,> 2 tấn,2.350000e+08
498,Hyundai,Veloster,2012,70000,Đã sử dụng,Tự động,Xăng,Đang cập nhật,Coupe (2 cửa),5,Bảo hành hãng,> 1 tấn,> 2 tấn,3.190000e+08


In [ ]:
result.isnull().sum()

Hãng                    0
Dòng xe                 0
Năm sản xuất            0
Số Km đã đi            73
Tình trạng              0
Hộp số                  0
Nhiên liệu              0
Xuất xứ                 0
Kiểu dáng              77
Số chỗ                 51
Chính sách bảo hành     0
Trọng lượng             0
Trọng tải               0
Giá                    0
dtype: int64

In [ ]:
result.to_csv('Car_price.csv')